In [1]:
import os
from KEYS.secrets import OPENAI_API_KEY, LANGCHAIN_API_KEY
from langchain_openai import OpenAI

### Setting up Environmental Variables

In [7]:
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="Langchain_Learn1"

### Model Input Output

#### LLM Models

In [3]:
llm = OpenAI()

In [4]:
response = llm.invoke("How many wonders of the world are in Nepal")
print(response)

?

There are no official "wonders of the world" designated in Nepal, but there are certainly many amazing sights and cultural landmarks that could be considered wonders. Some examples include:

1. Mount Everest - the highest mountain in the world and a marvel of nature
2. Pashupatinath Temple - a sacred Hindu temple complex and UNESCO World Heritage Site
3. Chitwan National Park - a diverse and stunning natural reserve, home to many rare and endangered species
4. Boudhanath Stupa - one of the largest Buddhist stupas in the world and an important pilgrimage site
5. Durbar Square in Kathmandu - a complex of ancient palaces, temples, and courtyards, showcasing Nepal's rich history and architecture
6. Annapurna Circuit - a popular trekking route that offers breathtaking views of the Himalayas
7. Lumbini - the birthplace of Buddha and a significant pilgrimage site for Buddhists
8. Bhaktapur - a well-preserved medieval city with beautiful architecture and traditional Newari culture
9. Sagarm

In [5]:
for chunk in llm.stream("Where were the 2012 Olympics held?"):
    print(chunk, end="", flush=True)



The 2012 Olympics were held in London, England.

#### Chat Models

In [7]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI()

In [8]:
from langchain.schema.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="You are Micheal Jordan."),
    HumanMessage(content="Which shoe manufacturer are you associated with?"),
]
response = chat.invoke(messages)
print(response.content)

I am associated with Nike.


### Prompts

In [2]:
from langchain.prompts import PromptTemplate

# Simple prompt with placeholders
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# Filling placeholders to create a prompt
filled_prompt = prompt_template.format(adjective="funny", content="robots")
print(filled_prompt)

Tell me a funny joke about robots.


In [3]:
from langchain.prompts import ChatPromptTemplate

# Defining a chat prompt with various roles
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

# Formatting the chat prompt
formatted_messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
for message in formatted_messages:
    print(message)

content='You are a helpful AI bot. Your name is Bob.'
content='Hello, how are you doing?'
content="I'm doing well, thanks!"
content='What is your name?'


### Output Parsers


#### PydanticOutputParsers

In [5]:
from typing import List
from langchain_openai import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [12]:
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

In [13]:
# Define your desired data structure using Pydantic
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

In [14]:
# Set up a PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=Joke)

# Create a prompt with format instructions
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [15]:
# Define a query to prompt the language model
query = "Tell me a joke."

# Combine prompt, model, and parser to get structured output
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": query})

# Parse the output using the parser
parsed_result = parser.invoke(output)

# The result is a structured object
print(parsed_result)

setup='Why did the tomato turn red?' punchline='Because it saw the salad dressing!'


#### SimpleJsonOutputParser

In [17]:
from langchain.output_parsers.json import SimpleJsonOutputParser

# Create a JSON prompt
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with `birthdate` and `birthplace` key that answers the following question: {question}"
)

# Initialize the JSON parser
json_parser = SimpleJsonOutputParser()

# Create a chain with the prompt, model, and parser
json_chain = json_prompt | model | json_parser

# Stream through the results
result_list = list(json_chain.stream({"question": "When and where was Elon Musk born?"}))

# The result is a list of JSON-like dictionaries
print(result_list[-1])

{'birthdate': 'June 28, 1971', 'birthplace': 'Pretoria, South Africa'}


### CommaSeparatedListOutputParser


In [19]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

# Initialize the parser
output_parser = CommaSeparatedListOutputParser()

# Create format instructions
format_instructions = output_parser.get_format_instructions()

# Create a prompt to request a list
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# Define a query to prompt the model
query = "English Premier League Teams"

# Generate the output
output = model(prompt.format(subject=query))

# Parse the output using the parser
parsed_result = output_parser.parse(output)

# The result is a list of items
print(parsed_result)


/home/aayush/Documents/Fusemachines/Langchain_Upskilling/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


['1. Manchester City\n2. Manchester United\n3. Liverpool\n4. Chelsea\n5. Arsenal']


### DatetimeOutputParser

In [22]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain
from langchain_openai import OpenAI

# Initialize the DatetimeOutputParser
output_parser = DatetimeOutputParser()

# Create a prompt with format instructions
template = """
Answer the user's question:
{question}
{format_instructions}
"""


prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# Create a chain with the prompt and language model
llm = OpenAI()
chain = prompt | llm
# Define a query to prompt the model
query = "when did Neil Armstrong land on the moon in terms of GMT?"

# Run the chain
output = chain.invoke(query)

# Parse the output using the datetime parser
parsed_result = output_parser.parse(output)

# The result is a datetime object
print(parsed_result)

1969-07-20 20:17:40
